##### Copyright 2024 Google LLC.

In [18]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompting with an Apollo 11 transcript

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides a quick example of how to prompt Gemini 1.5 Pro using a text file. In this case, you'll use a 400 page transcript from [Apollo 11](https://www.nasa.gov/history/alsj/a11/a11trans.html).

In [19]:
!pip install -U -q google-generativeai

In [20]:
import google.generativeai as genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [21]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Download the transcript.

In [22]:
!wget https://storage.googleapis.com/generativeai-downloads/data/a11.txt

--2024-05-14 20:12:01--  https://storage.googleapis.com/generativeai-downloads/data/a11.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 173.194.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847790 (828K) [text/plain]
Saving to: ‘a11.txt.2’

a11.txt.2           100%[===================>] 827.92K  --.-KB/s    in 0.009s  

2024-05-14 20:12:01 (90.5 MB/s) - ‘a11.txt.2’ saved [847790/847790]



Prepare it for use in a prompt.

In [23]:
text_file_name = "a11.txt"
print(f"Uploading file...")
text_file = genai.upload_file(path=text_file_name)
print(f"Completed upload: {text_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/7w6ag8dszigf


## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI. We'll ask the model to find a few lighthearted moments.

In [24]:
prompt = "Find four lighthearted moments in this text file."

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

response = model.generate_content([prompt, text_file],
                                  request_options={"timeout": 600})
print(response.text)

This is a very technical document and doesn't contain many explicitly lighthearted moments, but here are four instances where humor can be found:

1. **00 00 03 36 CDR
Yes. They finally gave me a window to look out.** - Neil Armstrong's dry humor about finally getting a window to see out of is a bit of lightheartedness during a very serious procedure.

2. **00 05 20 31 CMP
If we're late in answering you, it's because we're munching sandwiches.** - This comment from Michael Collins shows the astronauts taking a moment to enjoy a snack, even amidst their mission duties.

3. **01 03 23 00 CC
(Laughing.) A lot of gray paint.** - The capsule communicator's laughter at Buzz Aldrin's question about a possible Navy term for the reflection of the sun on the ocean is a lighthearted moment of shared humor.

4. **02 08 03 53 CC
Roger. We haven't quite got that before the 50,000,000 TV audience, yet.** - This humorous remark by the capsule communicator in response to Buzz Aldrin's comment about his

## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [25]:
genai.delete_file(text_file.name)

## Learning more

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) here.